# DCIR Setup Leveraging EDB Configuration Format

## Import the required packages

In [1]:
import json
import os
import tempfile

from ansys.aedt.core import Hfss3dLayout
from ansys.aedt.core.downloads import download_file

from pyedb import Edb

AEDT_VERSION = "2024.2"
NG_MODE = False

Download the example BGA Package design.

In [2]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source=r"edb/BGA_Package.aedb", destination=temp_folder.name)

## Load example layout

In [3]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

PyAEDT INFO: Logger is initialized in EDB.


PyAEDT INFO: legacy v0.35.0


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database BGA_Package.aedb Opened in 2024.2


PyAEDT INFO: Cell BGA_Package Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


## Create config file

Define Component with solderballs.

In [4]:
cfg_components = [
    {
        "reference_designator": "FCHIP",
        "part_type": "other",
        "solder_ball_properties": {
            "shape": "cylinder",
            "diameter": "0.1mm",
            "height": "0.085mm",
            "chip_orientation": "chip_up",
        },
        "port_properties": {
            "reference_offset": 0,
            "reference_size_auto": False,
            "reference_size_x": 0,
            "reference_size_y": 0,
        },
    },
    {
        "reference_designator": "BGA",
        "part_type": "io",
        "solder_ball_properties": {
            "shape": "cylinder",
            "diameter": "0.45mm",
            "height": "0.45mm",
            "chip_orientation": "chip_down",
        },
        "port_properties": {
            "reference_offset": 0,
            "reference_size_auto": False,
            "reference_size_x": 0,
            "reference_size_y": 0,
        },
    },
]

Define Pin Groups on Components.

In [5]:
cfg_pin_groups = [
    {"name": "BGA_VSS", "reference_designator": "BGA", "net": "VSS"},
    {"name": "BGA_VDD", "reference_designator": "BGA", "net": "VDD"},
]

Define sources.

In [6]:
cfg_sources = [
    {
        "name": "FCHIP_Current",
        "reference_designator": "FCHIP",
        "type": "current",
        "magnitude": 2,
        "distributed": True,
        "positive_terminal": {"net": "VDD"},
        "negative_terminal": {"nearest_pin": {"reference_net": "VSS", "search_radius": 5e-3}},
    },
    {
        "name": "BGA_Voltage",
        "reference_designator": "BGA",
        "type": "voltage",
        "magnitude": 1,
        "positive_terminal": {"pin_group": "BGA_VDD"},
        "negative_terminal": {"pin_group": "BGA_VSS"},
    },
]

Define DCIR setup.

In [7]:
cfg_setups = [
    {
        "name": "siwave_dc",
        "type": "siwave_dc",
        "dc_slider_position": 1,
        "dc_ir_settings": {"export_dc_thermal_data": True},
    }
]

Create final configuration.

In [8]:
cfg = {
    "components": cfg_components,
    "sources": cfg_sources,
    "pin_groups": cfg_pin_groups,
    "setups": cfg_setups,
}

Create the config file.

In [9]:
file_json = os.path.join(temp_folder.name, "edb_configuration.json")
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

## Apply Config file

Apply configuration to the example layout

In [10]:
edbapp.configuration.load(config_file=file_json)
edbapp.configuration.run()

True

Save and close EDB.

In [11]:
edbapp.save()
edbapp.close()

True

The configured EDB file is saved in a temp folder.

In [12]:
print(temp_folder.name)

C:\Users\ansys\AppData\Local\Temp\tmpmhkza63h.ansys


## Load edb into HFSS 3D Layout.

In [13]:
h3d = Hfss3dLayout(edbapp.edbpath, version=AEDT_VERSION, non_graphical=NG_MODE, new_desktop=True)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: PyAEDT version 0.14.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_1b62da3b-1a8f-4739-979a-c37d699ef12d.log is enabled.


...

PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64


PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 3136.


PyAEDT INFO: EDB folder C:\Users\ansys\AppData\Local\Temp\tmpmhkza63h.ansys\edb/BGA_Package.aedb has been imported to project BGA_Package


PyAEDT INFO: Active Design set to 0;BGA_Package


PyAEDT INFO: Aedt Objects correctly read


Solve.

In [14]:
h3d.analyze(setup="siwave_dc")

PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/HFSS 3D Layout Design correctly changed.


PyAEDT INFO: Solving design setup siwave_dc


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/HFSS 3D Layout Design correctly changed.


PyAEDT INFO: Design setup siwave_dc solved correctly in 0.0h 0.0m 25.0s


True

Plot insertion loss.

In [15]:
h3d.post.create_fieldplot_layers_nets(layers_nets=[["VDD_C1", "VDD"]], quantity="Voltage", setup="siwave_dc")

PyAEDT INFO: Parsing C:/Users/ansys/AppData/Local/Temp/tmpmhkza63h.ansys/edb/BGA_Package.aedt.


PyAEDT INFO: File C:/Users/ansys/AppData/Local/Temp/tmpmhkza63h.ansys/edb/BGA_Package.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.032442331314086914


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


...

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database BGA_Package.aedb Opened in 2024.2


PyAEDT INFO: Cell BGA_Package Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


Shut Down Electronics Desktop

In [16]:
h3d.close_desktop()

PyAEDT INFO: Desktop has been released and closed.


True

All project files are saved in the folder ``temp_file.dir``. If you've run this example as a Jupyter notebook you
can retrieve those project files.